# Project 3: Multivariate Regression Analysis and Gradient Boosting with Lowess and XGB

## Abstract:

In this notebook I present Multivariate Regression Analysis and Gradient Boosting with Lowess, Random Forest Regression, a home-made boosted Lowess model, and  the Extreme Gradient Boosting method (XGB). I apply each method to an auto-MPG dataset, and a Boston Housing Prices dataset. I present conceptualizations of gradient boosting, and use KFold cross-validation to test each model. I found that for the MPG data, Extreme Gradient Boosting was the best option by a significant margin, and for Bston Housing data I found that the homemade gradient boosting method performed the best by a smaller margin.

In [49]:
#Imports
import numpy as np
import pandas as pd
from scipy.linalg import lstsq
from scipy.interpolate import interp1d, LinearNDInterpolator, NearestNDInterpolator
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import xgboost as xgb 

## The Data

These data sets are both quite small, and I am using them as two dimensional. For the boston housing data, I have selected the 'cmedv', or the median sale price per zipcode, as our target and, since this is multivariate regression, I am using all 14 features. For the 'cars' dataset, I've selected the miles-per-gallon of each car as the target, and again are using all of the features. Below are examples of the data. Neither is a good candidate for a regular linear regression.

I did very little preprocessing on this data. It came quite well curated, so the only thing I needed to do was scale the features, for which I used scikitlearn's StandardScalar, and do a train test split (75% train, 25% test).

In [52]:
#Importing and Displaying Data

cars = pd.read_csv('Data/cars.csv')
boston = pd.read_csv('Data/Boston_Housing_Prices.csv')

Cars = np.concatenate([cars[['ENG','CYL','WGT']].values, cars['MPG'].values.reshape(-1,1)], axis=1)
Boston = np.concatenate([boston[['tract', 'longitude', 'latitude', 'crime', 'residential', 'industrial', 'nox', 'rooms', 'older', 
                                'distance', 'highway', 'tax', 'ptratio', 'lstat']].values, boston['cmedv'].values.reshape(-1,1)], axis=1)

model_xgb = xgb.XGBRegressor(objective ='reg:squarederror',n_estimators=100,reg_lambda=20,alpha=1,gamma=10,max_depth=3)

print("Boston Housing Data:")
display(boston.head())
print("Auto MPG Data:")
display(cars.head())

Boston Housing Data:


,town,tract,longitude,latitude,crime,residential,industrial,river,nox,rooms,older,distance,highway,tax,ptratio,lstat,cmedv
0,Nahant,2011,-70.955002,42.255001,0.00632,18.0,2.31,no,0.538,6.575,65.199997,4.0900,1,296,15.300000,4.98,24.000000
1,Swampscott,2021,-70.949997,42.287498,0.02731,0.0,7.07,no,0.469,6.421,78.900002,4.9671,2,242,17.799999,9.14,21.600000
2,Swampscott,2022,-70.935997,42.283001,0.02729,0.0,7.07,no,0.469,7.185,61.099998,4.9671,2,242,17.799999,4.03,34.700001
3,Marblehead,2031,-70.928001,42.292999,0.03237,0.0,2.18,no,0.458,6.998,45.799999,6.0622,3,222,18.700001,2.94,33.400002
4,Marblehead,2032,-70.921997,42.298000,0.06905,0.0,2.18,no,0.458,7.147,54.200001,6.0622,3,222,18.700001,5.33,36.200001


Auto MPG Data:


,MPG,CYL,ENG,WGT
0,18.0,8,307.0,3504
1,15.0,8,350.0,3693
2,18.0,8,318.0,3436
3,16.0,8,304.0,3433
4,17.0,8,302.0,3449


## The Methods and Models

### Gradient Boosting
Put simply, to "boost" a known regression model is to train n other regression models to predict the original model's residual errors, then add these predicted residuals to the original models test output. This results in a reduced error in the test output.

### Extreme Gradient Boosting (XGB)

In [61]:
#Tricubic Kernel, Lowess, and Boosted Lowess definitions

def Tricubic(x):
  if len(x.shape) == 1:
    x = x.reshape(-1,1)
  d = np.sqrt(np.sum(x**2,axis=1))
  return np.where(d>1,0,70/81*(1-d**3)**3)

def lw_reg(X, y, xnew, kern, tau, intercept):
    # tau is called bandwidth K((x-x[i])/(2*tau))
    n = len(X) # the number of observations
    yest = np.zeros(n)

    if len(y.shape)==1: # here we make column vectors
      y = y.reshape(-1,1)

    if len(X.shape)==1:
      X = X.reshape(-1,1)
    
    if intercept:
      X1 = np.column_stack([np.ones((len(X),1)),X])
    else:
      X1 = X

    w = np.array([kern((X - X[i])/(2*tau)) for i in range(n)]) # here we compute n vectors of weights

    #Looping through all X-points
    for i in range(n):          
        W = np.diag(w[:,i])
        b = np.transpose(X1).dot(W).dot(y)
        A = np.transpose(X1).dot(W).dot(X1)
        #A = A + 0.001*np.eye(X1.shape[1]) # if we want L2 regularization
        #theta = linalg.solve(A, b) # A*theta = b
        beta, res, rnk, s = lstsq(A, b)
        yest[i] = np.dot(X1[i],beta)
    if X.shape[1]==1:
      f = interp1d(X.flatten(),yest,fill_value='extrapolate')
    else:
      f = LinearNDInterpolator(X, yest)
    output = f(xnew) # the output may have NaN's where the data points from xnew are outside the convex hull of X
    if sum(np.isnan(output))>0:
      g = NearestNDInterpolator(X,y.ravel()) 
      # output[np.isnan(output)] = g(X[np.isnan(output)])
      output[np.isnan(output)] = g(xnew[np.isnan(output)])
    return output

def boosted_lwr(X, y, xnew, kern, tau, intercept): 
  # we need decision trees
  # for training the boosted method we use X and y
  Fx = lw_reg(X,y,X,kern,tau,intercept) # we need this for training the Decision Tree
  # Now train the Decision Tree on y_i - F(x_i)
  new_y = y - Fx
  #model = DecisionTreeRegressor(max_depth=2, random_state=123)
  model = RandomForestRegressor(n_estimators=100,max_depth=2)
  #model = model_xgb
  model.fit(X,new_y)
  output = model.predict(xnew) + lw_reg(X,y,xnew,kern,tau,intercept)
  return output    

def rep_boosted_lwr(X, y, xtest, kern, tau, booster, nboost, intercept):
  yhat = lw_reg(X,y,X,kern,tau,intercept)
  yhat_test = lw_reg(X,y,xtest,kern,tau,intercept)
  lw_error = y - yhat
  for i in range(nboost):
    booster.fit(X,lw_error)
    yhat += booster.predict(X)
    yhat_test += booster.predict(xtest)
    lw_error = y - yhat
  return yhat_test

### Cross Validation and Results

In [62]:
#Cars Cross-Validation

mse_lwr = []
mse_blwr = []
mse_rf = []
mse_xgb = []

scale = StandardScaler()

for i in range(12345,12350):
  #print('Random State: ' + str(i))
  kf = KFold(n_splits=10,shuffle=True,random_state=i)
  # this is the random state cross-validation loop to make sure our results are real, not just the state being good/bad for a particular model
  j = 0
  for idxtrain, idxtest in kf.split(Cars[:,:2]):
    j += 1
    #Split the train and test data
    xtrain = Cars[:,:2][idxtrain]
    ytrain = Cars[:,-1][idxtrain]
    ytest = Cars[:,-1][idxtest]
    xtest = Cars[:,:2][idxtest]
    xtrain = scale.fit_transform(xtrain)
    xtest = scale.transform(xtest)
    #print('Split Number: ' + str(j))

    #Train and predict for LWR and boosted LWR
    yhat_lwr = lw_reg(xtrain,ytrain, xtest,Tricubic,tau=1.2,intercept=True)
    
    #Boosted LWR with repeated boosting
    booster_rf = RandomForestRegressor(n_estimators=100,max_depth=2)
    yhat_blwr = rep_boosted_lwr(xtrain,ytrain,xtest,Tricubic,1.2,booster_rf,2,True)

    #Train and predict with random forest
    model_rf = RandomForestRegressor(n_estimators=100,max_depth=3)
    model_rf.fit(xtrain,ytrain)
    yhat_rf = model_rf.predict(xtest)

    #Train and predict for XGB
    model_xgb.fit(xtrain,ytrain)
    yhat_xgb = model_xgb.predict(xtest)

    #Append each model's MSE
    mse_lwr.append(mse(ytest,yhat_lwr))
    mse_blwr.append(mse(ytest,yhat_blwr))
    mse_rf.append(mse(ytest,yhat_rf))
    mse_xgb.append(mse(ytest,yhat_xgb))

print('\n')
print('The Cross-validated Mean Squared Error for LWR is : '+str(np.mean(mse_lwr)))
print('The Cross-validated Mean Squared Error for BLWR is : '+str(np.mean(mse_blwr)))
print('The Cross-validated Mean Squared Error for RF is : '+str(np.mean(mse_rf)))
print('The Cross-validated Mean Squared Error for XGB is : '+str(np.mean(mse_xgb)))



The Cross-validated Mean Squared Error for LWR is : 18.74002057325897
The Cross-validated Mean Squared Error for BLWR is : 16.671541246819412
The Cross-validated Mean Squared Error for RF is : 17.13980908049434
The Cross-validated Mean Squared Error for XGB is : 16.313673784632016


In [55]:
#Boston Cross-Validation

mse_lwr = []
mse_blwr = []
mse_rf = []
mse_xgb = []

scale = StandardScaler()

for i in range(12345,12350):
  #print('Random State: ' + str(i))
  kf = KFold(n_splits=10,shuffle=True,random_state=i)
  # this is the random state cross-validation loop to make sure our results are real, not just the state being good/bad for a particular model
  j = 0
  for idxtrain, idxtest in kf.split(Cars[:,:13]):
    j += 1
    #Split the train and test data
    xtrain = Cars[:,:13][idxtrain]
    ytrain = Cars[:,-1][idxtrain]
    ytest = Cars[:,-1][idxtest]
    xtest = Cars[:,:13][idxtest]
    xtrain = scale.fit_transform(xtrain)
    xtest = scale.transform(xtest)
    #print('Split Number: ' + str(j))

    #Train and predict for LWR and boosted LWR
    yhat_lwr = lw_reg(xtrain,ytrain, xtest,Tricubic,tau=1.2,intercept=True)
    
    #Boosted LWR with repeated boosting
    booster_rf = RandomForestRegressor(n_estimators=100,max_depth=2)
    yhat_blwr = rep_boosted_lwr(xtrain,ytrain,xtest,Tricubic,1.2,booster_rf,2,True)

    #Train and predict with random forest
    model_rf = RandomForestRegressor(n_estimators=100,max_depth=3)
    model_rf.fit(xtrain,ytrain)
    yhat_rf = model_rf.predict(xtest)

    #Train and predict for XGB
    model_xgb.fit(xtrain,ytrain)
    yhat_xgb = model_xgb.predict(xtest)

    #Append each model's MSE
    mse_lwr.append(mse(ytest,yhat_lwr))
    mse_blwr.append(mse(ytest,yhat_blwr))
    mse_rf.append(mse(ytest,yhat_rf))
    mse_xgb.append(mse(ytest,yhat_xgb))

print('\n')
print('The Cross-validated Mean Squared Error for LWR is : '+str(np.mean(mse_lwr)))
print('The Cross-validated Mean Squared Error for BLWR is : '+str(np.mean(mse_blwr)))
print('The Cross-validated Mean Squared Error for RF is : '+str(np.mean(mse_rf)))
print('The Cross-validated Mean Squared Error for XGB is : '+str(np.mean(mse_xgb)))



The Cross-validated Mean Squared Error for LWR is : 0.3587898269590638
The Cross-validated Mean Squared Error for BLWR is : 0.35878982695906286
The Cross-validated Mean Squared Error for RF is : 0.6216629815656345
The Cross-validated Mean Squared Error for XGB is : 0.5754137982172873
